In [2]:
import pyreadr
from pathlib import Path
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [7]:
##########################################################################
# load city df 
##########################################################################
result = pyreadr.read_r('/data/users/laura.owen/extremes/heatwaves/HadUKGrid/dur-clim/coords/UK_top30_cities.Rda')
city_df = result['city_df']

##########################################################################
# read in MSdata01 for London to test
##########################################################################
xco = 148
yco = 44
city_name = "London"
data_path = Path(f"/data/users/laura.owen/extremes/heatwaves/HadUKGrid/dur-clim/CPM5km/v2/UK/01/MakeStationary/MSdata01") / f"ukgd_cpm85_5k_x{xco}y{yco}_MSdata01.RData"
data01 = pyreadr.read_r(data_path)
data01 = data01["data01"] 






In [ ]:
##########################################################################
# format df
##########################################################################
#add date columns
data01['year'] = np.floor(data01['time']).astype(int)
data01['date'] = pd.to_datetime(data01['year'].astype(str) + '-01-01')
#Add the day-of-year offset (subtract 1 because DOY=1 means Jan 1)
data01['date'] = data01['date'] + pd.to_timedelta(data01['doy'] - 1, unit='D')
data01['month'] = data01['date'].dt.month
data01['month_name'] = data01['date'].dt.month_name()

#get only mods
print(data01.head())
#data01 = data01[data01['class'] == "obs"]
data01 = data01[data01['class'] == "mod"]

print(data01.head())


           x         time     stime  doy      sdoy  isobs class      gmst  \
0  11.933993  1960.002732 -0.500000    1  0.002732    1.0   obs -1.003301   
1   8.884717  1960.005464 -0.499977    2  0.005464    1.0   obs -1.003301   
2  11.406516  1960.008197 -0.499955    3  0.008197    1.0   obs -1.003301   
3  12.488321  1960.010929 -0.499932    4  0.010929    1.0   obs -1.003301   
4   9.682284  1960.013661 -0.499910    5  0.013661    1.0   obs -1.003300   

      uqgam  threshold     u  year       date  month month_name  
0  0.902317  12.662822  0.94  1960 1960-01-01      1    January  
1  0.645759  12.642253  0.94  1960 1960-01-02      1    January  
2  0.868147  12.621096  0.94  1960 1960-01-03      1    January  
3  0.935276  12.599455  0.94  1960 1960-01-04      1    January  
4  0.725268  12.577434  0.94  1960 1960-01-05      1    January  
           x         time     stime  doy      sdoy  isobs class      gmst  \
0  11.933993  1960.002732 -0.500000    1  0.002732    1.0   obs 

In [52]:
fig = px.line(
    data01,
    x='date',
    y='threshold',
    title='Threshold Over Time',
    labels={'date': 'Date', 'threshold': 'Threshold'}
)

fig.update_traces(
    hovertemplate='<b>Date:</b> %{x|%Y-%m-%d}<br><b>Threshold:</b> %{y:.2f}'
)

fig.show()


In [53]:
fig = px.line(
    data01,
    x='date',
    y='threshold',
    color='month_name',
    title='London Threshold with Overall Trend',
    labels={'threshold': 'Threshold', 'date': 'Date', 'month_name': 'Month'}
)

# Customize the hover to show full date
fig.update_traces(
    hovertemplate='<b>Date:</b> %{x|%Y-%m-%d}<br>' +
                  '<b>Threshold:</b> %{y:.2f}<br>' +
                  '<b>Month:</b> %{legendgroup}'
)

# Add the thick black line for the full data trend
fig.add_trace(
    go.Scatter(
        x=data01['date'],
        y=data01['threshold'],
        mode='lines',
        name='All Data',
        line=dict(color='black', width=2.5),
        showlegend=True,
        hovertemplate='<b>Date:</b> %{x|%Y-%m-%d}<br>' +
                      '<b>Threshold:</b> %{y:.2f}<br>' +
                      '<b>Month:</b> All'
    )
)

fig.show()

